# Xây dựng model sử dụng nn.Module

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/hoang/anaconda3/envs/deep_papers/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Lớp kế thừa từ Module
class Net(nn.Module):
    def __init__(self):
        # Gọi hàm khởi tạo của class parent
        super(Net, self).__init__()
        # Lớp convolution
        self.conv1 = nn.Conv2d(1, 6, 3) # (input_ch, output_ch, kernel_size)
        self.conv2 = nn.Conv2d(6, 16, 3) # (input_ch, output_ch, kernel_size)
        # Lớp fully connected
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    # Lan truyền thuận    
    def forward(self, x):
        # Sau khi đi qua conv1 thì x có 6 channels
        x = self.conv1(x)
        # Relu trả về x nếu x>0, trả về 0 nếu x <0
        x = F.relu(x)
        # Sử dụng max pooling, kernel kích thước 2x2
        x = F.max_pool2d(x, (2, 2))
        
        x = self.conv2(x)
        x = F.relu(x)
        # Viết là 2 thì max pooling tự hiểu là 2x2
        x = F.max_pool2d(x, 2)
        
        # Chuyển về dạng 1 chiều
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        

        return x
        
    def num_flat_features(self, x):
        """Đưa x có 2 chiều vào"""
        # Lấy ra dimension, bỏ đi dimension đầu
        size = x.size()[1:]
        num_features = 1
        # Nhân các chiều với nhau để ra số features
        for s in size:
            num_features *= s
        return num_features
    


## torchTensor.view 
Tương đương với reshape
Example:
```python
a = torch.arange(1, 17)  # a's shape is (16,)

a.view(4, 4) # output below
  1   2   3   4
  5   6   7   8
  9  10  11  12
 13  14  15  16
[torch.FloatTensor of size 4x4]

# Có thể hiểu chia thành 2 channel mỗi channel là 1 ma trận 2 hàng 4 cột
a.view(2, 2, 4) # output below
(0 ,.,.) = 
1   2   3   4
5   6   7   8

(1 ,.,.) = 
 9  10  11  12
13  14  15  16
[torch.FloatTensor of size 2x2x4]
```

In [14]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [15]:
# batch_size, channels, hàng, cột
input_image = torch.randn(1, 1, 32, 32)
output = net(input_image)
output.size()

torch.Size([1, 10])